# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.


In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

# **21_EYE**

In [ ]:
df_21_EYE = pd.read_csv('data/STData/21/21_EYE.csv')

In [ ]:
df_21_EYE.head()

In [ ]:
df_21_EYE.shape

In [ ]:
df_21_EYE.columns

In [ ]:
df_21_EYE.info()

In [ ]:
df_21_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_21_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_21_EYE['QuestionKey'].unique()

In [ ]:
df_21_EYE['Timestamp'] = pd.to_datetime(df_21_EYE['Timestamp'])

In [ ]:
df_21_EYE.head(3)

In [ ]:
df_21_EYE['QuestionKey'] = df_21_EYE['QuestionKey'].fillna('None')

In [ ]:
df_21_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_21_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_21_EYE.isnull().sum()

In [ ]:
df_21_EYE.dropna(inplace=True)

In [ ]:
df_21_EYE.head()

In [ ]:
df_21_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_21_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_21_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_21_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_21_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_21_EYE['ET_ValidityRight'].unique()

In [ ]:
df_21_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_21_EYE['ET_ValidityLeft'].value_counts().index, y=df_21_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_21_EYE['ET_ValidityRight'].value_counts().index, y=df_21_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_21_EYE['ET_ValidityLeft'] = df_21_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_21_EYE['ET_ValidityRight'] = df_21_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_21_EYE.head(3)

In [ ]:
df_21_EYE.describe()

In [ ]:
df_21_EYE[df_21_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_21_EYE[df_21_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_21_EYE[df_21_EYE['ET_ValidityLeft'] == 1].shape[0] / df_21_EYE.shape[0]

In [ ]:
df_21_EYE[df_21_EYE['ET_ValidityRight'] == 1].shape[0] / df_21_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_21_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_21_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_21_EYE[df_21_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_21_EYE[df_21_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_21_EYE[df_21_EYE['ET_PupilLeft'] == -1].shape[0] / df_21_EYE.shape[0]

In [ ]:
df_21_EYE[df_21_EYE['ET_PupilRight'] == -1].shape[0] / df_21_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_21_EYE[df_21_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_21_EYE[df_21_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_21_EYE['ET_PupilLeft_validity'] = df_21_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_21_EYE['ET_PupilRight_validity'] = df_21_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_21_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_21_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_21_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_21_EYE['ET_PupilLeft_validity'] = df_21_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_21_EYE['ET_PupilRight_validity'] = df_21_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_21_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_21_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_21_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_21_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_21_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_21_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_21_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_21_EYE['ET_PupilLeft_validity'] = df_21_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_21_EYE['ET_PupilRight_validity'] = df_21_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_21_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_21_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_21_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_21_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_21_EYE['Timestamp'], df_21_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_21_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_21_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_21_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_21_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_21_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_21_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_21_EYE[col] = df_21_EYE[col].fillna(df_21_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_21_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_21_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_21_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_21_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_21_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_21_EYE['Timestamp'], df_21_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_21_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_21_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_21_EYE)
plt.show()

# **22_EYE**

In [ ]:
df_22_EYE = pd.read_csv('data/STData/22/22_EYE.csv')

In [ ]:
df_22_EYE.head()

In [ ]:
df_22_EYE.shape

In [ ]:
df_22_EYE.columns

In [ ]:
df_22_EYE.info()

In [ ]:
df_22_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_22_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_22_EYE['QuestionKey'].unique()

In [ ]:
df_22_EYE['Timestamp'] = pd.to_datetime(df_22_EYE['Timestamp'])

In [ ]:
df_22_EYE.head(3)

In [ ]:
df_22_EYE['QuestionKey'] = df_22_EYE['QuestionKey'].fillna('None')

In [ ]:
df_22_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_22_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_22_EYE.isnull().sum()

In [ ]:
df_22_EYE.dropna(inplace=True)

In [ ]:
df_22_EYE.head()

In [ ]:
df_22_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_22_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_22_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_22_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_22_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_22_EYE['ET_ValidityRight'].unique()

In [ ]:
df_22_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_22_EYE['ET_ValidityLeft'].value_counts().index, y=df_22_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_22_EYE['ET_ValidityRight'].value_counts().index, y=df_22_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_22_EYE['ET_ValidityLeft'] = df_22_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_22_EYE['ET_ValidityRight'] = df_22_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_22_EYE.head(3)

In [ ]:
df_22_EYE.describe()

In [ ]:
df_22_EYE[df_22_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_22_EYE[df_22_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_22_EYE[df_22_EYE['ET_ValidityLeft'] == 1].shape[0] / df_22_EYE.shape[0]

In [ ]:
df_22_EYE[df_22_EYE['ET_ValidityRight'] == 1].shape[0] / df_22_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_22_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_22_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_22_EYE[df_22_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_22_EYE[df_22_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_22_EYE[df_22_EYE['ET_PupilLeft'] == -1].shape[0] / df_22_EYE.shape[0]

In [ ]:
df_22_EYE[df_22_EYE['ET_PupilRight'] == -1].shape[0] / df_22_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_22_EYE[df_22_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_22_EYE[df_22_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_22_EYE['ET_PupilLeft_validity'] = df_22_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_22_EYE['ET_PupilRight_validity'] = df_22_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_22_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_22_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_22_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_22_EYE['ET_PupilLeft_validity'] = df_22_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_22_EYE['ET_PupilRight_validity'] = df_22_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_22_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_22_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_22_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_22_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_22_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_22_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_22_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_22_EYE['ET_PupilLeft_validity'] = df_22_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_22_EYE['ET_PupilRight_validity'] = df_22_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_22_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_22_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_22_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_22_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_22_EYE['Timestamp'], df_22_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_22_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_22_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_22_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_22_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_22_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_22_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_22_EYE[col] = df_22_EYE[col].fillna(df_22_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_22_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_22_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_22_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_22_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_22_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_22_EYE['Timestamp'], df_22_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_22_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_22_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_22_EYE)
plt.show()

# **23_EYE**

In [ ]:
df_23_EYE = pd.read_csv('data/STData/23/23_EYE.csv')

In [ ]:
df_23_EYE.head()

In [ ]:
df_23_EYE.shape

In [ ]:
df_23_EYE.columns

In [ ]:
df_23_EYE.info()

In [ ]:
df_23_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_23_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_23_EYE['QuestionKey'].unique()

In [ ]:
df_23_EYE['Timestamp'] = pd.to_datetime(df_23_EYE['Timestamp'])

In [ ]:
df_23_EYE.head(3)

In [ ]:
df_23_EYE['QuestionKey'] = df_23_EYE['QuestionKey'].fillna('None')

In [ ]:
df_23_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_23_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_23_EYE.isnull().sum()

In [ ]:
df_23_EYE.dropna(inplace=True)

In [ ]:
df_23_EYE.head()

In [ ]:
df_23_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_23_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_23_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_23_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_23_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_23_EYE['ET_ValidityRight'].unique()

In [ ]:
df_23_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_23_EYE['ET_ValidityLeft'].value_counts().index, y=df_23_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_23_EYE['ET_ValidityRight'].value_counts().index, y=df_23_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_23_EYE['ET_ValidityLeft'] = df_23_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_23_EYE['ET_ValidityRight'] = df_23_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_23_EYE.head(3)

In [ ]:
df_23_EYE.describe()

In [ ]:
df_23_EYE[df_23_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_23_EYE[df_23_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_23_EYE[df_23_EYE['ET_ValidityLeft'] == 1].shape[0] / df_23_EYE.shape[0]

In [ ]:
df_23_EYE[df_23_EYE['ET_ValidityRight'] == 1].shape[0] / df_23_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_23_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_23_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_23_EYE[df_23_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_23_EYE[df_23_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_23_EYE[df_23_EYE['ET_PupilLeft'] == -1].shape[0] / df_23_EYE.shape[0]

In [ ]:
df_23_EYE[df_23_EYE['ET_PupilRight'] == -1].shape[0] / df_23_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_23_EYE[df_23_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_23_EYE[df_23_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_23_EYE['ET_PupilLeft_validity'] = df_23_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_23_EYE['ET_PupilRight_validity'] = df_23_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_23_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_23_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_23_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_23_EYE['ET_PupilLeft_validity'] = df_23_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_23_EYE['ET_PupilRight_validity'] = df_23_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_23_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_23_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_23_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_23_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_23_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_23_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_23_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_23_EYE['ET_PupilLeft_validity'] = df_23_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_23_EYE['ET_PupilRight_validity'] = df_23_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_23_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_23_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_23_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_23_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_23_EYE['Timestamp'], df_23_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_23_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_23_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_23_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_23_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_23_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_23_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_23_EYE[col] = df_23_EYE[col].fillna(df_23_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_23_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_23_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_23_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_23_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_23_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_23_EYE['Timestamp'], df_23_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_23_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_23_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_23_EYE)
plt.show()

# **24_EYE**

In [ ]:
df_24_EYE = pd.read_csv('data/STData/24/24_EYE.csv')

In [ ]:
df_24_EYE.head()

In [ ]:
df_24_EYE.shape

In [ ]:
df_24_EYE.columns

In [ ]:
df_24_EYE.info()

In [ ]:
df_24_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_24_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_24_EYE['QuestionKey'].unique()

In [ ]:
df_24_EYE['Timestamp'] = pd.to_datetime(df_24_EYE['Timestamp'])

In [ ]:
df_24_EYE.head(3)

In [ ]:
df_24_EYE['QuestionKey'] = df_24_EYE['QuestionKey'].fillna('None')

In [ ]:
df_24_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_24_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_24_EYE.isnull().sum()

In [ ]:
df_24_EYE.dropna(inplace=True)

In [ ]:
df_24_EYE.head()

In [ ]:
df_24_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_24_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_24_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_24_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_24_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_24_EYE['ET_ValidityRight'].unique()

In [ ]:
df_24_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_24_EYE['ET_ValidityLeft'].value_counts().index, y=df_24_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_24_EYE['ET_ValidityRight'].value_counts().index, y=df_24_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_24_EYE['ET_ValidityLeft'] = df_24_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_24_EYE['ET_ValidityRight'] = df_24_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_24_EYE.head(3)

In [ ]:
df_24_EYE.describe()

In [ ]:
df_24_EYE[df_24_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_24_EYE[df_24_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_24_EYE[df_24_EYE['ET_ValidityLeft'] == 1].shape[0] / df_24_EYE.shape[0]

In [ ]:
df_24_EYE[df_24_EYE['ET_ValidityRight'] == 1].shape[0] / df_24_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_24_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_24_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_24_EYE[df_24_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_24_EYE[df_24_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_24_EYE[df_24_EYE['ET_PupilLeft'] == -1].shape[0] / df_24_EYE.shape[0]

In [ ]:
df_24_EYE[df_24_EYE['ET_PupilRight'] == -1].shape[0] / df_24_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_24_EYE[df_24_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_24_EYE[df_24_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_24_EYE['ET_PupilLeft_validity'] = df_24_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_24_EYE['ET_PupilRight_validity'] = df_24_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_24_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_24_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_24_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_24_EYE['ET_PupilLeft_validity'] = df_24_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_24_EYE['ET_PupilRight_validity'] = df_24_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_24_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_24_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_24_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_24_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_24_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_24_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_24_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_24_EYE['ET_PupilLeft_validity'] = df_24_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_24_EYE['ET_PupilRight_validity'] = df_24_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_24_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_24_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_24_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_24_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_24_EYE['Timestamp'], df_24_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_24_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_24_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_24_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_24_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_24_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_24_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_24_EYE[col] = df_24_EYE[col].fillna(df_24_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_24_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_24_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_24_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_24_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_24_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_24_EYE['Timestamp'], df_24_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_24_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_24_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_24_EYE)
plt.show()

# **25_EYE**

In [ ]:
df_25_EYE = pd.read_csv('data/STData/25/25_EYE.csv')

In [ ]:
df_25_EYE.head()

In [ ]:
df_25_EYE.shape

In [ ]:
df_25_EYE.columns

In [ ]:
df_25_EYE.info()

In [ ]:
df_25_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_25_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_25_EYE['QuestionKey'].unique()

In [ ]:
df_25_EYE['Timestamp'] = pd.to_datetime(df_25_EYE['Timestamp'])

In [ ]:
df_25_EYE.head(3)

In [ ]:
df_25_EYE['QuestionKey'] = df_25_EYE['QuestionKey'].fillna('None')

In [ ]:
df_25_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_25_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_25_EYE.isnull().sum()

In [ ]:
df_25_EYE.dropna(inplace=True)

In [ ]:
df_25_EYE.head()

In [ ]:
df_25_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_25_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_25_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_25_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_25_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_25_EYE['ET_ValidityRight'].unique()

In [ ]:
df_25_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_25_EYE['ET_ValidityLeft'].value_counts().index, y=df_25_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_25_EYE['ET_ValidityRight'].value_counts().index, y=df_25_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_25_EYE['ET_ValidityLeft'] = df_25_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_25_EYE['ET_ValidityRight'] = df_25_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_25_EYE.head(3)

In [ ]:
df_25_EYE.describe()

In [ ]:
df_25_EYE[df_25_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_25_EYE[df_25_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_25_EYE[df_25_EYE['ET_ValidityLeft'] == 1].shape[0] / df_25_EYE.shape[0]

In [ ]:
df_25_EYE[df_25_EYE['ET_ValidityRight'] == 1].shape[0] / df_25_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_25_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_25_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_25_EYE[df_25_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_25_EYE[df_25_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_25_EYE[df_25_EYE['ET_PupilLeft'] == -1].shape[0] / df_25_EYE.shape[0]

In [ ]:
df_25_EYE[df_25_EYE['ET_PupilRight'] == -1].shape[0] / df_25_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_25_EYE[df_25_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_25_EYE[df_25_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_25_EYE['ET_PupilLeft_validity'] = df_25_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_25_EYE['ET_PupilRight_validity'] = df_25_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_25_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_25_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_25_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_25_EYE['ET_PupilLeft_validity'] = df_25_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_25_EYE['ET_PupilRight_validity'] = df_25_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_25_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_25_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_25_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_25_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_25_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_25_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_25_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_25_EYE['ET_PupilLeft_validity'] = df_25_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_25_EYE['ET_PupilRight_validity'] = df_25_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_25_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_25_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_25_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_25_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_25_EYE['Timestamp'], df_25_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_25_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_25_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_25_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_25_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_25_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_25_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_25_EYE[col] = df_25_EYE[col].fillna(df_25_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_25_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_25_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_25_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_25_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_25_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_25_EYE['Timestamp'], df_25_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_25_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_25_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_25_EYE)
plt.show()

# **26_EYE**

In [ ]:
df_26_EYE = pd.read_csv('data/STData/26/26_EYE.csv')

In [ ]:
df_26_EYE.head()

In [ ]:
df_26_EYE.shape

In [ ]:
df_26_EYE.columns

In [ ]:
df_26_EYE.info()

In [ ]:
df_26_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_26_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_26_EYE['QuestionKey'].unique()

In [ ]:
df_26_EYE['Timestamp'] = pd.to_datetime(df_26_EYE['Timestamp'])

In [ ]:
df_26_EYE.head(3)

In [ ]:
df_26_EYE['QuestionKey'] = df_26_EYE['QuestionKey'].fillna('None')

In [ ]:
df_26_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_26_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_26_EYE.isnull().sum()

In [ ]:
df_26_EYE.dropna(inplace=True)

In [ ]:
df_26_EYE.head()

In [ ]:
df_26_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_26_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_26_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_26_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_26_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_26_EYE['ET_ValidityRight'].unique()

In [ ]:
df_26_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_26_EYE['ET_ValidityLeft'].value_counts().index, y=df_26_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_26_EYE['ET_ValidityRight'].value_counts().index, y=df_26_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_26_EYE['ET_ValidityLeft'] = df_26_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_26_EYE['ET_ValidityRight'] = df_26_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_26_EYE.head(3)

In [ ]:
df_26_EYE.describe()

In [ ]:
df_26_EYE[df_26_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_26_EYE[df_26_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_26_EYE[df_26_EYE['ET_ValidityLeft'] == 1].shape[0] / df_26_EYE.shape[0]

In [ ]:
df_26_EYE[df_26_EYE['ET_ValidityRight'] == 1].shape[0] / df_26_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_26_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_26_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_26_EYE[df_26_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_26_EYE[df_26_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_26_EYE[df_26_EYE['ET_PupilLeft'] == -1].shape[0] / df_26_EYE.shape[0]

In [ ]:
df_26_EYE[df_26_EYE['ET_PupilRight'] == -1].shape[0] / df_26_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_26_EYE[df_26_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_26_EYE[df_26_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_26_EYE['ET_PupilLeft_validity'] = df_26_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_26_EYE['ET_PupilRight_validity'] = df_26_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_26_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_26_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_26_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_26_EYE['ET_PupilLeft_validity'] = df_26_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_26_EYE['ET_PupilRight_validity'] = df_26_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_26_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_26_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_26_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_26_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_26_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_26_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_26_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_26_EYE['ET_PupilLeft_validity'] = df_26_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_26_EYE['ET_PupilRight_validity'] = df_26_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_26_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_26_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_26_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_26_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_26_EYE['Timestamp'], df_26_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_26_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_26_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_26_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_26_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_26_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_26_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_26_EYE[col] = df_26_EYE[col].fillna(df_26_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_26_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_26_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_26_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_26_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_26_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_26_EYE['Timestamp'], df_26_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_26_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_26_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_26_EYE)
plt.show()

# **27_EYE**

In [ ]:
df_27_EYE = pd.read_csv('data/STData/27/27_EYE.csv')

In [ ]:
df_27_EYE.head()

In [ ]:
df_27_EYE.shape

In [ ]:
df_27_EYE.columns

In [ ]:
df_27_EYE.info()

In [ ]:
df_27_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_27_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_27_EYE['QuestionKey'].unique()

In [ ]:
df_27_EYE['Timestamp'] = pd.to_datetime(df_27_EYE['Timestamp'])

In [ ]:
df_27_EYE.head(3)

In [ ]:
df_27_EYE['QuestionKey'] = df_27_EYE['QuestionKey'].fillna('None')

In [ ]:
df_27_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_27_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_27_EYE.isnull().sum()

In [ ]:
df_27_EYE.dropna(inplace=True)

In [ ]:
df_27_EYE.head()

In [ ]:
df_27_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_27_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_27_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_27_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_27_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_27_EYE['ET_ValidityRight'].unique()

In [ ]:
df_27_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_27_EYE['ET_ValidityLeft'].value_counts().index, y=df_27_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_27_EYE['ET_ValidityRight'].value_counts().index, y=df_27_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_27_EYE['ET_ValidityLeft'] = df_27_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_27_EYE['ET_ValidityRight'] = df_27_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_27_EYE.head(3)

In [ ]:
df_27_EYE.describe()

In [ ]:
df_27_EYE[df_27_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_27_EYE[df_27_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_27_EYE[df_27_EYE['ET_ValidityLeft'] == 1].shape[0] / df_27_EYE.shape[0]

In [ ]:
df_27_EYE[df_27_EYE['ET_ValidityRight'] == 1].shape[0] / df_27_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_27_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_27_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_27_EYE[df_27_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_27_EYE[df_27_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_27_EYE[df_27_EYE['ET_PupilLeft'] == -1].shape[0] / df_27_EYE.shape[0]

In [ ]:
df_27_EYE[df_27_EYE['ET_PupilRight'] == -1].shape[0] / df_27_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_27_EYE[df_27_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_27_EYE[df_27_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_27_EYE['ET_PupilLeft_validity'] = df_27_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_27_EYE['ET_PupilRight_validity'] = df_27_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_27_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_27_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_27_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_27_EYE['ET_PupilLeft_validity'] = df_27_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_27_EYE['ET_PupilRight_validity'] = df_27_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_27_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_27_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_27_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_27_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_27_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_27_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_27_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_27_EYE['ET_PupilLeft_validity'] = df_27_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_27_EYE['ET_PupilRight_validity'] = df_27_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_27_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_27_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_27_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_27_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_27_EYE['Timestamp'], df_27_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_27_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_27_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_27_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_27_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_27_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_27_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_27_EYE[col] = df_27_EYE[col].fillna(df_27_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_27_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_27_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_27_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_27_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_27_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_27_EYE['Timestamp'], df_27_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_27_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_27_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_27_EYE)
plt.show()

# **28_EYE**

In [ ]:
df_28_EYE = pd.read_csv('data/STData/28/28_EYE.csv')

In [ ]:
df_28_EYE.head()

In [ ]:
df_28_EYE.shape

In [ ]:
df_28_EYE.columns

In [ ]:
df_28_EYE.info()

In [ ]:
df_28_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_28_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_28_EYE['QuestionKey'].unique()

In [ ]:
df_28_EYE['Timestamp'] = pd.to_datetime(df_28_EYE['Timestamp'])

In [ ]:
df_28_EYE.head(3)

In [ ]:
df_28_EYE['QuestionKey'] = df_28_EYE['QuestionKey'].fillna('None')

In [ ]:
df_28_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_28_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_28_EYE.isnull().sum()

In [ ]:
df_28_EYE.dropna(inplace=True)

In [ ]:
df_28_EYE.head()

In [ ]:
df_28_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_28_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_28_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_28_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_28_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_28_EYE['ET_ValidityRight'].unique()

In [ ]:
df_28_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_28_EYE['ET_ValidityLeft'].value_counts().index, y=df_28_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_28_EYE['ET_ValidityRight'].value_counts().index, y=df_28_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_28_EYE['ET_ValidityLeft'] = df_28_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_28_EYE['ET_ValidityRight'] = df_28_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_28_EYE.head(3)

In [ ]:
df_28_EYE.describe()

In [ ]:
df_28_EYE[df_28_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_28_EYE[df_28_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_28_EYE[df_28_EYE['ET_ValidityLeft'] == 1].shape[0] / df_28_EYE.shape[0]

In [ ]:
df_28_EYE[df_28_EYE['ET_ValidityRight'] == 1].shape[0] / df_28_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_28_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_28_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_28_EYE[df_28_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_28_EYE[df_28_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_28_EYE[df_28_EYE['ET_PupilLeft'] == -1].shape[0] / df_28_EYE.shape[0]

In [ ]:
df_28_EYE[df_28_EYE['ET_PupilRight'] == -1].shape[0] / df_28_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_28_EYE[df_28_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_28_EYE[df_28_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_28_EYE['ET_PupilLeft_validity'] = df_28_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_28_EYE['ET_PupilRight_validity'] = df_28_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_28_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_28_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_28_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_28_EYE['ET_PupilLeft_validity'] = df_28_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_28_EYE['ET_PupilRight_validity'] = df_28_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_28_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_28_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_28_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_28_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_28_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_28_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_28_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_28_EYE['ET_PupilLeft_validity'] = df_28_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_28_EYE['ET_PupilRight_validity'] = df_28_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_28_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_28_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_28_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_28_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_28_EYE['Timestamp'], df_28_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_28_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_28_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_28_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_28_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_28_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_28_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_28_EYE[col] = df_28_EYE[col].fillna(df_28_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_28_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_28_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_28_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_28_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_28_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_28_EYE['Timestamp'], df_28_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_28_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_28_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_28_EYE)
plt.show()

# **29_EYE**

In [ ]:
df_29_EYE = pd.read_csv('data/STData/29/29_EYE.csv')

In [ ]:
df_29_EYE.head()

In [ ]:
df_29_EYE.shape

In [ ]:
df_29_EYE.columns

In [ ]:
df_29_EYE.info()

In [ ]:
df_29_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_29_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_29_EYE['QuestionKey'].unique()

In [ ]:
df_29_EYE['Timestamp'] = pd.to_datetime(df_29_EYE['Timestamp'])

In [ ]:
df_29_EYE.head(3)

In [ ]:
df_29_EYE['QuestionKey'] = df_29_EYE['QuestionKey'].fillna('None')

In [ ]:
df_29_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_29_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_29_EYE.isnull().sum()

In [ ]:
df_29_EYE.dropna(inplace=True)

In [ ]:
df_29_EYE.head()

In [ ]:
df_29_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_29_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_29_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_29_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_29_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_29_EYE['ET_ValidityRight'].unique()

In [ ]:
df_29_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_29_EYE['ET_ValidityLeft'].value_counts().index, y=df_29_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_29_EYE['ET_ValidityRight'].value_counts().index, y=df_29_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_29_EYE['ET_ValidityLeft'] = df_29_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_29_EYE['ET_ValidityRight'] = df_29_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_29_EYE.head(3)

In [ ]:
df_29_EYE.describe()

In [ ]:
df_29_EYE[df_29_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_29_EYE[df_29_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_29_EYE[df_29_EYE['ET_ValidityLeft'] == 1].shape[0] / df_29_EYE.shape[0]

In [ ]:
df_29_EYE[df_29_EYE['ET_ValidityRight'] == 1].shape[0] / df_29_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_29_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_29_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_29_EYE[df_29_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_29_EYE[df_29_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_29_EYE[df_29_EYE['ET_PupilLeft'] == -1].shape[0] / df_29_EYE.shape[0]

In [ ]:
df_29_EYE[df_29_EYE['ET_PupilRight'] == -1].shape[0] / df_29_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_29_EYE[df_29_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_29_EYE[df_29_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_29_EYE['ET_PupilLeft_validity'] = df_29_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_29_EYE['ET_PupilRight_validity'] = df_29_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_29_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_29_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_29_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_29_EYE['ET_PupilLeft_validity'] = df_29_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_29_EYE['ET_PupilRight_validity'] = df_29_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_29_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_29_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_29_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_29_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_29_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_29_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_29_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_29_EYE['ET_PupilLeft_validity'] = df_29_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_29_EYE['ET_PupilRight_validity'] = df_29_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_29_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_29_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_29_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_29_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_29_EYE['Timestamp'], df_29_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_29_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_29_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_29_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_29_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_29_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_29_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_29_EYE[col] = df_29_EYE[col].fillna(df_29_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_29_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_29_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_29_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_29_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_29_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_29_EYE['Timestamp'], df_29_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_29_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_29_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_29_EYE)
plt.show()

# **30_EYE**

In [ ]:
df_30_EYE = pd.read_csv('data/STData/30/30_EYE.csv')

In [ ]:
df_30_EYE.head()

In [ ]:
df_30_EYE.shape

In [ ]:
df_30_EYE.columns

In [ ]:
df_30_EYE.info()

In [ ]:
df_30_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_30_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_30_EYE['QuestionKey'].unique()

In [ ]:
df_30_EYE['Timestamp'] = pd.to_datetime(df_30_EYE['Timestamp'])

In [ ]:
df_30_EYE.head(3)

In [ ]:
df_30_EYE['QuestionKey'] = df_30_EYE['QuestionKey'].fillna('None')

In [ ]:
df_30_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_30_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_30_EYE.isnull().sum()

In [ ]:
df_30_EYE.dropna(inplace=True)

In [ ]:
df_30_EYE.head()

In [ ]:
df_30_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_30_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_30_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_30_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_30_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_30_EYE['ET_ValidityRight'].unique()

In [ ]:
df_30_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_30_EYE['ET_ValidityLeft'].value_counts().index, y=df_30_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_30_EYE['ET_ValidityRight'].value_counts().index, y=df_30_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_30_EYE['ET_ValidityLeft'] = df_30_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_30_EYE['ET_ValidityRight'] = df_30_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_30_EYE.head(3)

In [ ]:
df_30_EYE.describe()

In [ ]:
df_30_EYE[df_30_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_30_EYE[df_30_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_30_EYE[df_30_EYE['ET_ValidityLeft'] == 1].shape[0] / df_30_EYE.shape[0]

In [ ]:
df_30_EYE[df_30_EYE['ET_ValidityRight'] == 1].shape[0] / df_30_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_30_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_30_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_30_EYE[df_30_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_30_EYE[df_30_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_30_EYE[df_30_EYE['ET_PupilLeft'] == -1].shape[0] / df_30_EYE.shape[0]

In [ ]:
df_30_EYE[df_30_EYE['ET_PupilRight'] == -1].shape[0] / df_30_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_30_EYE[df_30_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_30_EYE[df_30_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_30_EYE['ET_PupilLeft_validity'] = df_30_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_30_EYE['ET_PupilRight_validity'] = df_30_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_30_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_30_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_30_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_30_EYE['ET_PupilLeft_validity'] = df_30_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_30_EYE['ET_PupilRight_validity'] = df_30_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_30_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_30_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_30_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_30_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_30_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_30_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_30_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_30_EYE['ET_PupilLeft_validity'] = df_30_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_30_EYE['ET_PupilRight_validity'] = df_30_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_30_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_30_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_30_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_30_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_30_EYE['Timestamp'], df_30_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_30_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_30_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_30_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_30_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_30_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_30_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_30_EYE[col] = df_30_EYE[col].fillna(df_30_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_30_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_30_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_30_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_30_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_30_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_30_EYE['Timestamp'], df_30_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_30_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_30_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_30_EYE)
plt.show()